# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Versions
Kippes - Do not use Tensorflow 1.3.0.  Use 1.1.0.
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [2]:
view_sentence_range = (0, 50)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 50:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.


Moe_Szyslak: Ah, isn't that nice. Now, there is a politician 

## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [3]:
import numpy as np
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function

    vocab = sorted(set(text))
    
    # create word-->ID and ID-->word dictionaries
    vocab_to_int = {word:idx for idx,word in enumerate(vocab)}
    int_to_vocab = dict(enumerate(vocab))

    return vocab_to_int, int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [4]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    
    f = lambda w: '||' + w + '||'
    
    token_dict = {
        '.' :  f('period'),
        ',' :  f('comma'),
        '"' :  f('quotation_mark'),
        ';' :  f('semicolon'),
        '!' :  f('exclamation_mark'),
        '?' :  f('question_mark'),
        '(' :  f('left_parenthesis'),
        ')' :  f('right_parenthesis'),
        #'-' :  f('dash'),
        '--' : f('em_dash'),
        '\n' : f('new_line')
    }
    
    return token_dict

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [5]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [6]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.1.0
Default GPU Device: /gpu:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [8]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    
    input = tf.placeholder(tf.int32, (None, None), name='input')
    targets = tf.placeholder(tf.int32, (None, None), name='targets')
    learning_rate = tf.placeholder(tf.float32, name='learning_rate')
    
    return input, targets, learning_rate

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [9]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
    
    # See Anna_KaRNNa notebook
    
    build_cell = lambda lstm_size: tf.contrib.rnn.BasicLSTMCell(lstm_size)
    
    NUM_LAYERS = 2
    
    lstm = tf.contrib.rnn.MultiRNNCell([build_cell(rnn_size) for _ in range(NUM_LAYERS)])
    
    initial_state = tf.identity(lstm.zero_state(batch_size, tf.float32), 'initial_state')
    
    return lstm, initial_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [10]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    
    # See Skip-Grams notebook
    
    embedding = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1, 1))
    embed = tf.nn.embedding_lookup(embedding, input_data)
    
    return embed

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [11]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    
    # See Anna_KaRNNa notebook
    
    outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    
    return outputs, tf.identity(final_state, "final_state")

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [12]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    
    embedded_input = get_embed(input_data, vocab_size, embed_dim)
    outputs, final_state = build_rnn(cell, embedded_input)
    logits = tf.contrib.layers.fully_connected(outputs, vocab_size, None)
    
    return logits, final_state

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [13]:
from pprint import pprint

def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    
    # Get the number of words per batch and number of batches we can make
    words_per_batch = batch_size * seq_length
    num_batches = len(int_text) // words_per_batch
    
    # Drop incomplete batches
    int_text = np.array(int_text[:words_per_batch * num_batches])
    
    # Create targets as a 1-word shifted version of int_text
    target_text = np.append(int_text[1:], np.array([int_text[0]]))
    
    # Reshape in rows that will split into batches
    int_text = int_text.reshape((batch_size, -1))
    target_text = target_text.reshape((batch_size, -1))
    
    input_batches = np.empty((num_batches, batch_size), dtype=object)
    target_batches = np.empty((num_batches, batch_size), dtype=object)
    
    # Create numpy array placeholders for everything
    rvalue = np.empty((num_batches, 2, batch_size, seq_length), dtype=object)
        
    for set_idx in range(0, len(int_text)):
        input_sets = int_text[set_idx].reshape((-1, seq_length))
        target_sets = target_text[set_idx].reshape((-1, seq_length))
        
        for b in range(0, len(input_sets)):
            rvalue[b][0][set_idx] = input_sets[b]
            rvalue[b][1][set_idx] = target_sets[b]
        
    return rvalue

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [14]:
# Number of Epochs
num_epochs = 100
# Batch Size
batch_size = 50
# RNN Size
rnn_size = 512
# Embedding Dimension Size
embed_dim = 1000
# Sequence Length
seq_length = 18 # Individual sentences to generate (12 words is average input)
# Learning Rate
learning_rate = 0.01
# Show stats for every n number of batches
show_every_n_batches = 5

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [15]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [16]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/76   train_loss = 8.821
Epoch   0 Batch    5/76   train_loss = 7.068
Epoch   0 Batch   10/76   train_loss = 6.545
Epoch   0 Batch   15/76   train_loss = 6.295
Epoch   0 Batch   20/76   train_loss = 6.142
Epoch   0 Batch   25/76   train_loss = 5.907
Epoch   0 Batch   30/76   train_loss = 5.961
Epoch   0 Batch   35/76   train_loss = 5.708
Epoch   0 Batch   40/76   train_loss = 5.826
Epoch   0 Batch   45/76   train_loss = 5.676
Epoch   0 Batch   50/76   train_loss = 5.433
Epoch   0 Batch   55/76   train_loss = 5.633
Epoch   0 Batch   60/76   train_loss = 5.451
Epoch   0 Batch   65/76   train_loss = 5.356
Epoch   0 Batch   70/76   train_loss = 5.473
Epoch   0 Batch   75/76   train_loss = 5.451
Epoch   1 Batch    4/76   train_loss = 5.123
Epoch   1 Batch    9/76   train_loss = 5.032
Epoch   1 Batch   14/76   train_loss = 5.315
Epoch   1 Batch   19/76   train_loss = 5.210
Epoch   1 Batch   24/76   train_loss = 5.046
Epoch   1 Batch   29/76   train_loss = 5.033
Epoch   1 

Epoch  12 Batch    3/76   train_loss = 2.809
Epoch  12 Batch    8/76   train_loss = 3.028
Epoch  12 Batch   13/76   train_loss = 3.079
Epoch  12 Batch   18/76   train_loss = 2.892
Epoch  12 Batch   23/76   train_loss = 2.899
Epoch  12 Batch   28/76   train_loss = 2.840
Epoch  12 Batch   33/76   train_loss = 2.867
Epoch  12 Batch   38/76   train_loss = 2.827
Epoch  12 Batch   43/76   train_loss = 2.863
Epoch  12 Batch   48/76   train_loss = 2.824
Epoch  12 Batch   53/76   train_loss = 2.990
Epoch  12 Batch   58/76   train_loss = 2.933
Epoch  12 Batch   63/76   train_loss = 2.868
Epoch  12 Batch   68/76   train_loss = 2.816
Epoch  12 Batch   73/76   train_loss = 2.702
Epoch  13 Batch    2/76   train_loss = 2.891
Epoch  13 Batch    7/76   train_loss = 2.800
Epoch  13 Batch   12/76   train_loss = 2.761
Epoch  13 Batch   17/76   train_loss = 2.887
Epoch  13 Batch   22/76   train_loss = 2.875
Epoch  13 Batch   27/76   train_loss = 2.982
Epoch  13 Batch   32/76   train_loss = 2.863
Epoch  13 

Epoch  24 Batch    6/76   train_loss = 1.359
Epoch  24 Batch   11/76   train_loss = 1.498
Epoch  24 Batch   16/76   train_loss = 1.495
Epoch  24 Batch   21/76   train_loss = 1.417
Epoch  24 Batch   26/76   train_loss = 1.554
Epoch  24 Batch   31/76   train_loss = 1.480
Epoch  24 Batch   36/76   train_loss = 1.504
Epoch  24 Batch   41/76   train_loss = 1.370
Epoch  24 Batch   46/76   train_loss = 1.286
Epoch  24 Batch   51/76   train_loss = 1.466
Epoch  24 Batch   56/76   train_loss = 1.469
Epoch  24 Batch   61/76   train_loss = 1.302
Epoch  24 Batch   66/76   train_loss = 1.356
Epoch  24 Batch   71/76   train_loss = 1.312
Epoch  25 Batch    0/76   train_loss = 1.369
Epoch  25 Batch    5/76   train_loss = 1.359
Epoch  25 Batch   10/76   train_loss = 1.311
Epoch  25 Batch   15/76   train_loss = 1.266
Epoch  25 Batch   20/76   train_loss = 1.206
Epoch  25 Batch   25/76   train_loss = 1.293
Epoch  25 Batch   30/76   train_loss = 1.379
Epoch  25 Batch   35/76   train_loss = 1.356
Epoch  25 

Epoch  36 Batch    9/76   train_loss = 1.161
Epoch  36 Batch   14/76   train_loss = 1.183
Epoch  36 Batch   19/76   train_loss = 1.172
Epoch  36 Batch   24/76   train_loss = 1.100
Epoch  36 Batch   29/76   train_loss = 1.031
Epoch  36 Batch   34/76   train_loss = 1.032
Epoch  36 Batch   39/76   train_loss = 1.131
Epoch  36 Batch   44/76   train_loss = 1.065
Epoch  36 Batch   49/76   train_loss = 1.030
Epoch  36 Batch   54/76   train_loss = 1.173
Epoch  36 Batch   59/76   train_loss = 1.123
Epoch  36 Batch   64/76   train_loss = 1.039
Epoch  36 Batch   69/76   train_loss = 1.138
Epoch  36 Batch   74/76   train_loss = 1.077
Epoch  37 Batch    3/76   train_loss = 1.026
Epoch  37 Batch    8/76   train_loss = 1.063
Epoch  37 Batch   13/76   train_loss = 1.023
Epoch  37 Batch   18/76   train_loss = 1.000
Epoch  37 Batch   23/76   train_loss = 1.129
Epoch  37 Batch   28/76   train_loss = 0.952
Epoch  37 Batch   33/76   train_loss = 0.974
Epoch  37 Batch   38/76   train_loss = 1.053
Epoch  37 

Epoch  48 Batch   12/76   train_loss = 0.604
Epoch  48 Batch   17/76   train_loss = 0.567
Epoch  48 Batch   22/76   train_loss = 0.641
Epoch  48 Batch   27/76   train_loss = 0.638
Epoch  48 Batch   32/76   train_loss = 0.654
Epoch  48 Batch   37/76   train_loss = 0.638
Epoch  48 Batch   42/76   train_loss = 0.627
Epoch  48 Batch   47/76   train_loss = 0.581
Epoch  48 Batch   52/76   train_loss = 0.620
Epoch  48 Batch   57/76   train_loss = 0.627
Epoch  48 Batch   62/76   train_loss = 0.581
Epoch  48 Batch   67/76   train_loss = 0.561
Epoch  48 Batch   72/76   train_loss = 0.541
Epoch  49 Batch    1/76   train_loss = 0.544
Epoch  49 Batch    6/76   train_loss = 0.572
Epoch  49 Batch   11/76   train_loss = 0.644
Epoch  49 Batch   16/76   train_loss = 0.624
Epoch  49 Batch   21/76   train_loss = 0.558
Epoch  49 Batch   26/76   train_loss = 0.643
Epoch  49 Batch   31/76   train_loss = 0.553
Epoch  49 Batch   36/76   train_loss = 0.583
Epoch  49 Batch   41/76   train_loss = 0.565
Epoch  49 

Epoch  60 Batch   15/76   train_loss = 1.288
Epoch  60 Batch   20/76   train_loss = 1.316
Epoch  60 Batch   25/76   train_loss = 1.325
Epoch  60 Batch   30/76   train_loss = 1.379
Epoch  60 Batch   35/76   train_loss = 1.366
Epoch  60 Batch   40/76   train_loss = 1.280
Epoch  60 Batch   45/76   train_loss = 1.320
Epoch  60 Batch   50/76   train_loss = 1.414
Epoch  60 Batch   55/76   train_loss = 1.416
Epoch  60 Batch   60/76   train_loss = 1.395
Epoch  60 Batch   65/76   train_loss = 1.410
Epoch  60 Batch   70/76   train_loss = 1.454
Epoch  60 Batch   75/76   train_loss = 1.509
Epoch  61 Batch    4/76   train_loss = 1.369
Epoch  61 Batch    9/76   train_loss = 1.447
Epoch  61 Batch   14/76   train_loss = 1.489
Epoch  61 Batch   19/76   train_loss = 1.420
Epoch  61 Batch   24/76   train_loss = 1.472
Epoch  61 Batch   29/76   train_loss = 1.356
Epoch  61 Batch   34/76   train_loss = 1.419
Epoch  61 Batch   39/76   train_loss = 1.381
Epoch  61 Batch   44/76   train_loss = 1.484
Epoch  61 

Epoch  72 Batch   18/76   train_loss = 0.467
Epoch  72 Batch   23/76   train_loss = 0.456
Epoch  72 Batch   28/76   train_loss = 0.407
Epoch  72 Batch   33/76   train_loss = 0.424
Epoch  72 Batch   38/76   train_loss = 0.454
Epoch  72 Batch   43/76   train_loss = 0.415
Epoch  72 Batch   48/76   train_loss = 0.413
Epoch  72 Batch   53/76   train_loss = 0.436
Epoch  72 Batch   58/76   train_loss = 0.386
Epoch  72 Batch   63/76   train_loss = 0.484
Epoch  72 Batch   68/76   train_loss = 0.407
Epoch  72 Batch   73/76   train_loss = 0.383
Epoch  73 Batch    2/76   train_loss = 0.407
Epoch  73 Batch    7/76   train_loss = 0.393
Epoch  73 Batch   12/76   train_loss = 0.354
Epoch  73 Batch   17/76   train_loss = 0.364
Epoch  73 Batch   22/76   train_loss = 0.397
Epoch  73 Batch   27/76   train_loss = 0.399
Epoch  73 Batch   32/76   train_loss = 0.399
Epoch  73 Batch   37/76   train_loss = 0.365
Epoch  73 Batch   42/76   train_loss = 0.359
Epoch  73 Batch   47/76   train_loss = 0.334
Epoch  73 

Epoch  84 Batch   21/76   train_loss = 0.218
Epoch  84 Batch   26/76   train_loss = 0.230
Epoch  84 Batch   31/76   train_loss = 0.191
Epoch  84 Batch   36/76   train_loss = 0.239
Epoch  84 Batch   41/76   train_loss = 0.214
Epoch  84 Batch   46/76   train_loss = 0.204
Epoch  84 Batch   51/76   train_loss = 0.227
Epoch  84 Batch   56/76   train_loss = 0.236
Epoch  84 Batch   61/76   train_loss = 0.199
Epoch  84 Batch   66/76   train_loss = 0.184
Epoch  84 Batch   71/76   train_loss = 0.214
Epoch  85 Batch    0/76   train_loss = 0.218
Epoch  85 Batch    5/76   train_loss = 0.227
Epoch  85 Batch   10/76   train_loss = 0.198
Epoch  85 Batch   15/76   train_loss = 0.205
Epoch  85 Batch   20/76   train_loss = 0.186
Epoch  85 Batch   25/76   train_loss = 0.187
Epoch  85 Batch   30/76   train_loss = 0.227
Epoch  85 Batch   35/76   train_loss = 0.212
Epoch  85 Batch   40/76   train_loss = 0.192
Epoch  85 Batch   45/76   train_loss = 0.193
Epoch  85 Batch   50/76   train_loss = 0.227
Epoch  85 

Epoch  96 Batch   24/76   train_loss = 0.221
Epoch  96 Batch   29/76   train_loss = 0.192
Epoch  96 Batch   34/76   train_loss = 0.216
Epoch  96 Batch   39/76   train_loss = 0.211
Epoch  96 Batch   44/76   train_loss = 0.215
Epoch  96 Batch   49/76   train_loss = 0.204
Epoch  96 Batch   54/76   train_loss = 0.196
Epoch  96 Batch   59/76   train_loss = 0.212
Epoch  96 Batch   64/76   train_loss = 0.223
Epoch  96 Batch   69/76   train_loss = 0.220
Epoch  96 Batch   74/76   train_loss = 0.198
Epoch  97 Batch    3/76   train_loss = 0.164
Epoch  97 Batch    8/76   train_loss = 0.227
Epoch  97 Batch   13/76   train_loss = 0.192
Epoch  97 Batch   18/76   train_loss = 0.218
Epoch  97 Batch   23/76   train_loss = 0.204
Epoch  97 Batch   28/76   train_loss = 0.195
Epoch  97 Batch   33/76   train_loss = 0.187
Epoch  97 Batch   38/76   train_loss = 0.212
Epoch  97 Batch   43/76   train_loss = 0.205
Epoch  97 Batch   48/76   train_loss = 0.203
Epoch  97 Batch   53/76   train_loss = 0.221
Epoch  97 

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [17]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [18]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [19]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    
    loaded_input = loaded_graph.get_tensor_by_name('input:0')
    loaded_initial_state = loaded_graph.get_tensor_by_name('initial_state:0')
    loaded_final_state = loaded_graph.get_tensor_by_name('final_state:0')
    loaded_probs = loaded_graph.get_tensor_by_name('probs:0')
    
    return loaded_input, loaded_initial_state, loaded_final_state, loaded_probs


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [20]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    
    # Use Numpy to generate a random sample from the 1-D array
    words = list(int_to_vocab.values())
    picked_word = np.random.choice(list(int_to_vocab.values()), p=probabilities)

    return picked_word


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [21]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

INFO:tensorflow:Restoring parameters from ./save
moe_szyslak:(to little man) come on, jer. open up. be a pal. remember moe, you guys who are the president," hey," they did to be back.
carl_carlson: oh, hey, make you him?
carl_carlson: no, it's a little dank.
homer_simpson: hey, guys, you were been that right, but i don't feel so great girl, kid.
moe_szyslak: yeah! / you know, if it isn't mr. here's the hell...
homer_simpson:(forecast noise) you're all losers!
homer_simpson:(nods) yeah, but thank you go!
moe_szyslak: oh, the looks like that in the car.
moe_szyslak: just this is a channel six tip.
moe_szyslak:(excited) oh, my. times have.
lenny_leonard: was it easy!
krusty_the_clown:(don't the old are all my friends?
homer_simpson:(sadly) i can't. i've got some of that i don't know get back to my old face that i just use some tanking it.


# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the begging of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.